In [192]:
%load_ext autoreload
%autoreload 2

import sys
import os
import joblib

import pandas as pd
import numpy as np

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [193]:
data_dir = "/Users/andst/synology/phd/datasets/clustering/reproducing/Event"

In [194]:
torch.ones((2,2)).flatten()

tensor([1., 1., 1., 1.])

In [206]:
vec = joblib.load(os.path.join(data_dir, "Event_clu_init_300_1.00.pkl"))

In [208]:
vec.shape

(69, 300)

In [195]:
with open(os.path.join(data_dir, "Event_word2wid.txt")) as f:
    event2id = json.load(f)

In [197]:
event2id

{'2serious': 1,
 '3novic': 2,
 '9new': 3,
 'aaj': 4,
 'aajtv': 5,
 'aamaq': 6,
 'aamirliaquat': 7,
 'aap': 8,
 'aapk': 9,
 'ab': 10,
 'abba': 11,
 'abbassiya': 12,
 'abc': 13,
 'abcnew': 14,
 'abdeslam': 15,
 'abduct': 16,
 'abdul': 17,
 'ablaz': 18,
 'aboard': 19,
 'abrini': 20,
 'absolut': 21,
 'absurd': 22,
 'abt': 23,
 'abu': 24,
 'abus': 25,
 'ac': 26,
 'acc': 27,
 'acceler': 28,
 'accept': 29,
 'access': 30,
 'accid': 31,
 'accommod': 32,
 'accomplic': 33,
 'account': 34,
 'accus': 35,
 'achakzai': 36,
 'act': 37,
 'actagainstpak': 38,
 'action': 39,
 'activ': 40,
 'activist': 41,
 'actual': 42,
 'ad': 43,
 'adamawa': 44,
 'adan': 45,
 'add': 46,
 'addit': 47,
 'address': 48,
 'adel': 49,
 'aden': 50,
 'administr': 51,
 'admit': 52,
 'advanc': 53,
 'advic': 54,
 'advis': 55,
 'advoc': 56,
 'aerial': 57,
 'af': 58,
 'affair': 59,
 'affect': 60,
 'affili': 61,
 'afford': 62,
 'afg': 63,
 'afgh': 64,
 'afghan': 65,
 'afghani': 66,
 'afghanistan': 67,
 'afp': 68,
 'afraid': 69,
 'afr

In [94]:
vec = joblib.load(os.path.join(data_dir, "Event_word2vec_300.pkl"))
with open(os.path.join(data_dir, "Event_word2wid.txt")) as f:
    event2id = json.load(f)

word_embs = np.zeros((len(event2id), vec.get("break").shape[0]))
word_embs.shape
for key, val in event2id.items():
    word_embs[val - 1] = vec.get(key)
word_embs = torch.from_numpy(word_embs)

In [70]:
m = 0
for line in lines:
    if len(line["tokenids"]) > m:
        m = len(line["tokenids"])
m

18

In [91]:
import json

with open(os.path.join(data_dir, "Event_docarr.txt")) as f:
    lines = f.readlines()
    lines = [json.loads(l) for l in lines]

padded_length = 0
for line in lines:
    if len(line["tokenids"]) > padded_length:
        padded_length = len(line["tokenids"])
print(padded_length)
samples = []
masks = []
labels = []

for line in lines:
    labels.append(line["topic"])
    ids = line["tokenids"]
    ids = np.array([i - 1 for i in ids])
    samples.append(torch.from_numpy(ids))
    mask = np.zeros((padded_length,))
    mask[0:len(ids)] = 1
    masks.append(mask)
    
labels = torch.from_numpy(np.array(labels))
samples = torch.nn.utils.rnn.pad_sequence(samples).T
masks = torch.from_numpy(np.array(masks))

assert masks.shape == samples.shape

18


In [92]:
samples[0:10]

tensor([[ 419, 2676, 2454, 2848, 2947, 1506, 2608, 1424, 3095,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 419, 2681, 1093,  528, 2946,  250,  507, 2454,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 419, 2965,  234, 2946,  250, 1506, 1453, 1724, 2969, 2085,  576,    0,
            0,    0,    0,    0,    0,    0],
        [1453, 2946,  250, 2676, 2965,  234, 2901, 2454, 1453,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [2676,  234,  526, 2946,  250, 3261, 2618, 3160, 2840,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 419, 1453, 2946,  250, 2676, 2965,  234, 2901, 2454,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [1451, 2454, 3261, 2676,  866, 2848, 2946,  250,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 419, 1556, 2681, 3261, 2946,  250,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    

In [89]:
masks.shape

torch.Size([26619, 0])

In [142]:
from adv_clustering.loader import get_dataloader


word_embs, train_loader = get_dataloader(data_dir=data_dir)

In [181]:
train_loader.dataset.labels

tensor([ 0.,  0.,  0.,  ..., 68., 68., 68.])

In [182]:
word_embs.shape

torch.Size([3314, 300])

In [147]:
from adv_clustering.model import ARL


model = ARL(word_vecs=word_embs, num_clusters=68)

i = 0
for batch in train_loader:
    print(i)
    i += 1
    
    labels = batch.pop("labels")
    d, d_reconstructed, d_neg = model(**batch)
    if i is not None:
        break


0
torch.Size([64, 300, 18]) torch.Size([64, 18])


In [145]:
model.parameters()

<generator object Module.parameters at 0x14e6bf190>

In [189]:
from adv_clustering.trainer import Trainer


model = ARL(word_vecs=word_embs, num_clusters=64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t = Trainer(model=model, device=device)
t.train(data_loader=train_loader, lr=1e-3, num_epochs=100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:18<00:00, 22.04it/s]


Loss:  -0.9497871


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:04<00:00, 90.14it/s]


{'acc': 0.13001991059017995, 'nmi': 0.1702980351993078, 'ami': 0.1532821942776451, 'ari': 0.045795328720022066}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:19<00:00, 20.81it/s]


Loss:  -0.94103986


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:04<00:00, 90.95it/s]


{'acc': 0.13005747774146287, 'nmi': 0.17020085647180005, 'ami': 0.15316939032614438, 'ari': 0.04593576482226357}


 30%|███████████████████████████████████████████████████████████████▌                                                                                                                                                       | 123/416 [00:05<00:14, 20.74it/s]


KeyboardInterrupt: 

In [163]:
t.evaluate(train_loader)



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:05<00:00, 77.01it/s]


{'acc': 0.19760321574814982, 'nmi': 0.4224931538035421, 'ami': 0.38646250805477306, 'ari': 0.13068880017751477}


In [22]:
import torch 

torch.ones((3, 6)).unsqueeze(1).repeat(1, 15, 1).shape

torch.Size([3, 15, 6])

In [29]:
torch.cosine_similarity(torch.ones((2,2, 4)), torch.ones((2,2,4)), dim=2)

tensor([[1., 1.],
        [1., 1.]])

In [16]:
print(d.shape, d_reconstructed.shape, d_neg.shape)

torch.Size([2, 40]) torch.Size([2, 40]) torch.Size([2, 16, 40])


In [35]:
from adv_clustering.trainer import J1_loss

J1_loss(d, d_reconstructed, d_neg, margin=1)

tensor(2.8672, grad_fn=<AddBackward0>)